# N3 fine-mapping example (Python)

In [1]:
import numpy as np
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr, data
from rpy2.robjects import numpy2ri
susie = importr('susieR')

In [2]:
conversion_rules = numpy2ri.converter + ro.default_converter
with (conversion_rules).context():   
    ro.r['set.seed'](1)
    N3 = data(susie).fetch('N3finemapping')['N3finemapping']
    N3_names = {v:i for i,v in enumerate(N3.names)}
    n = N3[N3_names['X']].dim[0]

In [3]:
b = N3[N3_names['true_coef']]
np.where(np.array(b)[:,0] != 0)

(array([402, 652, 772]),)

In [4]:
with (conversion_rules).context():   
    sumstats = susie.univariate_regression(
        N3[N3_names['X']],
        np.array(N3[N3_names['Y']])[:,0]
    )
    z_scores = np.array(sumstats[0])/np.array(sumstats[1])
    R = np.corrcoef(N3[N3_names['X']], rowvar=False)
    fitted_rss1 = susie.susie_rss(
        bhat=sumstats[0], 
        shat=sumstats[1], 
        R=R,
        n=n,
        var_y=np.var(np.array(N3[N3_names['Y']])[:,0]),
        L=10,
        estimate_residual_variance=True
    )
    print(ro.r.summary(fitted_rss1)[1])

R[write to console]: HINT: For estimate_residual_variance = TRUE, please check that R is the "in-sample" LD matrix; that is, the correlation matrix obtained using the exact same data matrix X that was used for the other summary statistics. Also note, when covariates are included in the univariate regressions that produced the summary statistics, also consider removing these effects from X before computing R.



  cs cs_log10bf cs_avg_r2 cs_min_r2
1  2   4.033879 1.0000000 1.0000000
2  1   6.744085 0.9634847 0.9634847
3  3   3.461470 0.9293299 0.7545197
                                                                                                     variable
1                                                                                                         653
2                                                                                                     773,777
3 362,365,372,373,374,379,381,383,384,386,387,388,389,391,392,396,397,398,399,400,401,403,404,405,407,408,415



In [5]:
with (conversion_rules).context():   
    fitted = susie.susie(N3[N3_names['X']], 
                         np.array(N3[N3_names['Y']])[:,0], 
                         L=10
                        )
    print(ro.r['all.equal'](fitted[-2], fitted_rss1[-1]))

[1] TRUE



In [6]:
with (conversion_rules).context():   
    fitted_rss2 = susie.susie_rss(
        z=z_scores,
        R=R,
        n=n,  
        var_y=np.var(np.array(N3[N3_names['Y']])[:,0]),
        L=10,
        estimate_residual_variance=True
    )


R[write to console]: HINT: For estimate_residual_variance = TRUE, please check that R is the "in-sample" LD matrix; that is, the correlation matrix obtained using the exact same data matrix X that was used for the other summary statistics. Also note, when covariates are included in the univariate regressions that produced the summary statistics, also consider removing these effects from X before computing R.

R[write to console]: WARNING: XtX is not symmetric; forcing XtX to be symmetric by replacing XtX with (XtX + t(XtX))/2

R[write to console]: Error in Xty - s$XtXr : non-conformable arrays



RRuntimeError: Error in Xty - s$XtXr : non-conformable arrays


In [7]:
print(ro.r.sessionInfo())

R version 4.4.2 (2024-10-31)
Platform: x86_64-conda-linux-gnu
Running under: Red Hat Enterprise Linux 8.6 (Ootpa)

Matrix products: default
BLAS/LAPACK: /hpfs/userws/chiouj02/software/conda_envs/susie_rpy2/lib/libopenblasp-r0.3.29.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: America/New_York
tzcode source: system (glibc)

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] susieR_0.12.35

loaded via a namespace (and not attached):
 [1] crayon_1.5.3        mixsqp_0.3-54       vctrs_0.6.5        
 [4] cli_3.6.4           rlang_1.1.5         generics